<a href="https://colab.research.google.com/github/Yashvishe13/Deep-Learning/blob/main/Age_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/laxmimerit/Image-Dataset-for-Age-Prediction.git

Cloning into 'Image-Dataset-for-Age-Prediction'...
remote: Enumerating objects: 47107, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 47107 (delta 0), reused 2 (delta 0), pack-reused 47104
Receiving objects: 100% (47107/47107), 1.37 GiB | 31.24 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (24110/24110), done.


In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 97 kB/s 
     |████████████████████████████████| 6.8 MB 33.9 MB/s 
     |████████████████████████████████| 981 kB 40.3 MB/s 
     |████████████████████████████████| 263 kB 47.3 MB/s 
     |████████████████████████████████| 1.9 MB 44.9 MB/s 
     |████████████████████████████████| 1.2 MB 51.7 MB/s 
     |████████████████████████████████| 468 kB 39.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 47.8 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=1530a4eb82f436b4c2e3c61117fa40a9f4d37ccd8b5c56138de15c76aecada3d
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=878cbd277ee46cb7aa6f8fac2d02692cea30772eb30a3b9757a1e6b367cf2301
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [ ]:
%matplotlib inline
import os
import ktrain
from ktrain import vision as vis

In [ ]:
import re
pattern = r'([^/]+)_\d+_\d+_\d+.jpg$'
p = re.compile(pattern)
r = p.search('100_0_0_20170112213500903.jpg')
print(r.group(1))

100


In [ ]:
DATADIR = '/content/Image-Dataset-for-Age-Prediction/images'

In [ ]:
(train_data, test_data, preproc) = vis.images_from_fname(DATADIR, pattern = pattern, is_regression=True, random_state=42)

/usr/local/lib/python3.7/dist-packages/ktrain/vision/data.py:736: UserWarning: Could not extract target for /content/Image-Dataset-for-Age-Prediction/images/61_3_20170109150557335.jpg -  skipping this file
  warnings.warn('Could not extract target for %s -  skipping this file'% (fname))
/usr/local/lib/python3.7/dist-packages/ktrain/vision/data.py:736: UserWarning: Could not extract target for /content/Image-Dataset-for-Age-Prediction/images/24_0_1_20170116220224657 .jpg -  skipping this file
  warnings.warn('Could not extract target for %s -  skipping this file'% (fname))
/usr/local/lib/python3.7/dist-packages/ktrain/vision/data.py:736: UserWarning: Could not extract target for /content/Image-Dataset-for-Age-Prediction/images/61_1_20170109142408075.jpg -  skipping this file
  warnings.warn('Could not extract target for %s -  skipping this file'% (fname))
/usr/local/lib/python3.7/dist-packages/ktrain/vision/data.py:736: UserWarning: Could not extract target for /content/Image-Dataset-fo

['label']
       label
12304   29.0
6161     1.0
19807   24.0
21202   32.0
14941   21.0
['label']
       label
16139   25.0
14926    1.0
8979    26.0
12077   61.0
14568   37.0
Found 21690 validated image filenames.
Found 2411 validated image filenames.


In [ ]:
model = vis.image_regression_model('pretrained_resnet50', train_data=train_data, val_data=test_data)

The normalization scheme has been changed for use with a pretrained_resnet50 model. If you decide to use a different model, please reload your dataset with a ktrain.vision.data.images_from* function.

Is Multi-Label? False
Is Regression? True
94781440/94765736 [==============================] - 1s 0us/step
pretrained_resnet50 model created.


In [ ]:
learner = ktrain.get_learner(model= model, train_data= train_data, val_data=test_data, batch_size=64)

In [ ]:
learner.fit_onecycle(1e-4, 2)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/2
339/339 [==============================] - 437s 1s/step - loss: 394.0998 - mae: 14.7458 - val_loss: 180.7426 - val_mae: 10.1307
Epoch 2/2
339/339 [==============================] - 401s 1s/step - loss: 175.3920 - mae: 10.1274 - val_loss: 156.7900 - val_mae: 9.5738


In [ ]:
learner.freeze(15)

learner.fit_onecycle(1e-4, 2)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
test_data.filenames[0:50]

In [ ]:
def show_prediction(fname):
    fname = DATADIR + '/' + fname
    pred = round(predictor.predict_filename(fname)[0])
    if pred <= 0:
        pred += (-pred + 1)
    actual_age = int(p.search(fname).group(1))
    vis.show_image(fname)
    print("Predicted Age: %s | Actual Age: %s" % (pred, actual_age))

In [ ]:
show_prediction('35_0_1_20170113185408776.jpg')

In [ ]:
def real_time_prediction(fname):
    pred = round(predictor.predict_filename(fname)[0])
    vis.show_image(fname)
    print("Predicted Age: %s" % pred)

In [ ]:
real_time_prediction('photo.jpg')

In [ ]:
predictor.save('/content/age_prediction_model')

In [ ]:
!zip -r /content/age_predictor.zip /content/age_prediction_model/